In [ ]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import pandas as pd
import numpy as np
from scipy.stats import (
    bootstrap,
    gmean,
)
from IPython.display import display

from duet.process import *
from duet.constants import *
from common import *

df_raw = load_raw()
unique_suites = df_raw[RF.suite].unique()
df_raw.describe()

# Timeline

In [ ]:
# Look at first X minutes of a host
host = df_raw["hostname"].iloc[0]
host_start = df_raw[df_raw[AF.hostname] == host][RF.start].min()
df = df_raw[
    (
        (df_raw[AF.hostname] == host)
        & (df_raw[RF.start] < host_start + pd.DateOffset(minutes=30))
    )
]
if df.shape[0] < 1000:
    px.timeline(
        df,
        x_start=RF.start,
        x_end=RF.end,
        y=RF.pair,
        color=RF.type,
        hover_data=[RF.benchmark, RF.time],
    ).show()
    px.timeline(
        df,
        x_start=RF.start,
        x_end=RF.end,
        y=RF.iteration,
        color=RF.type,
        hover_data=[RF.benchmark, RF.time],
    ).show()
    px.scatter(
        df,
        x=RF.iteration,
        y=RF.time_ns,
        color=RF.pair,
        facet_col=RF.benchmark,
    ).show()
else:
    print(f"Data too large to display properly {df.shape[0]}")

In [ ]:
df = (
    df_raw.groupby(PAIR_ID_COL + ARTIFACT_COL)
    .agg(start=(RF.start, min), end=(RF.end, max))
    .reset_index()
)
px.timeline(
    df,
    x_start="start",
    x_end="end",
    y=AF.hostname,
    color="type",
    hover_data=[RF.suite, RF.benchmark, RF.pair, RF.runid],
)

# Run/Iteration count and Iteration time

In [ ]:
df_input = df_raw
df_input["prep"] = "raw"

df_prep = preprocess_data(df_raw)
df_prep["prep"] = "prep"
df_input = pd.concat([df_input, df_prep])

df_itercount = (
    df_input.groupby(ARTIFACT_COL + BENCHMARK_ID_COL + ["prep"])[RF.iteration]
    .count()
    .reset_index()
)
df_runcount = (
    df_input.groupby(ARTIFACT_COL + BENCHMARK_ID_COL)
    .agg(run_count=(RF.runid, "nunique"))
    .reset_index()
)

for suite in unique_suites:
    fig = px.bar(
        df_runcount[df_runcount[RF.suite] == suite],
        x=RF.benchmark,
        y="run_count",
        color=DF.env,
        facet_row=RF.type,
        barmode="group",
        title=f"Benchmark Run Count - {suite}",
    )
    fig.update_xaxes(categoryorder="category ascending")
    fig.show()

    fig = px.bar(
        df_itercount[df_itercount[RF.suite] == suite],
        x=RF.benchmark,
        y=RF.iteration,
        color="prep",
        facet_row=RF.type,
        facet_col=DF.env,
        barmode="group",
        title=f"Benchmark Iteration Count - {suite}",
    )
    fig.update_xaxes(categoryorder="category ascending")
    fig.show()

    fig = px.box(
        df_input[df_input[RF.suite] == suite],
        y=RF.time,
        x=RF.benchmark,
        color="prep",
        facet_row=RF.type,
        facet_col=DF.env,
        title=f"Benchmark Iteration Time - {suite}",
    )
    fig.update_xaxes(categoryorder="category ascending")
    fig.update_layout(height=1000)
    fig.show()

# Relative standard deviation

In [ ]:
df = preprocess_data(df_raw)

df = (
    df.groupby(BENCHMARK_ENV_COL)
    .agg(
        time_count=(RF.time_ns, len),
        time_mean=(RF.time_ns, "mean"),
        time_var=(RF.time_ns, "var"),
        time_std=(RF.time_ns, "std"),
    )
    .reset_index()
)
df["time_std_relative"] = df["time_std"] / df["time_mean"]

display(df)

df[color_type] = df[DF.env] + ":" + df[RF.type]
df = df.sort_values(by=color_type)

for suite in unique_suites:
    fig = px.bar(
        df[df[RF.suite] == suite],
        x=RF.benchmark,
        y="time_std_relative",
        barmode="group",
        color=color_type,
        title=f"Benchmark Time Relative Deviation - {suite}",
    )
    fig.update_xaxes(categoryorder="category ascending")
    fig.show()